In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from keras.models import Sequential, Model
from keras.layers import Input, Dense, Flatten, Dropout, Activation, Lambda, Permute, Reshape
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

from sklearn.datasets import load_files       
from keras.utils import np_utils

from tensorflow import keras

from glob import glob

import cv2

import matplotlib.pyplot as plt
from tqdm import tqdm

# load dataset


In [ ]:
base_path = '../input/deepfake-and-real-images/Dataset/'
image_gen = ImageDataGenerator(
    rescale=1./255.
)
batch_size = 128
train_flow = image_gen.flow_from_directory(
    base_path + 'Train/',
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode = "categorical"
)
valid_flow = image_gen.flow_from_directory(
    base_path + 'Validation/',
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode = "categorical"
)
test_flow = image_gen.flow_from_directory(
    base_path + 'Test/',
    target_size=(256, 256),
    batch_size=1,
    shuffle = False,
    class_mode = "categorical"
)

In [ ]:
print(test_flow[13][1])

In [ ]:
valid_flow[0][1]

# train_flow?

# Custom Architecture

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.layers import BatchNormalization
"""
    Propsoed CNN architecture.
    
"""

model = Sequential()

# Pamameters Initialization
input_shape = (256,256,3)
activation = 'relu'
padding = 'same'
droprate = 0.1
epsilon=0.001

model = Sequential()
model.add(BatchNormalization(input_shape=input_shape))
model.add(Conv2D(filters=16, kernel_size=3, activation=activation, padding=padding))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization(epsilon=epsilon))


model.add(Conv2D(filters=32, kernel_size=3, activation=activation, padding=padding))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization(epsilon=epsilon))
model.add(Dropout(droprate))

model.add(Conv2D(filters=64, kernel_size=3, activation=activation, padding=padding))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization(epsilon=epsilon))
model.add(Dropout(droprate))

model.add(Conv2D(filters=128, kernel_size=3, activation=activation, padding=padding))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization(epsilon=epsilon))
model.add(Dropout(droprate))

model.add(Conv2D(filters=256, kernel_size=3, activation=activation, padding=padding))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization(epsilon=epsilon))
model.add(Dropout(droprate))

model.add(Conv2D(filters=512, kernel_size=3, activation=activation, padding=padding))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization(epsilon=epsilon))
model.add(Dropout(droprate))

# model.add(Conv2D(filters=512, kernel_size=3, activation=activation, padding=padding))
# model.add(MaxPooling2D(pool_size=2))
# model.add(BatchNormalization(epsilon=epsilon))
# model.add(Dropout(droprate))

# model.add(GlobalAveragePooling2D())
model.add(Flatten())
# model.add(Dense(256, kernel_initializer='glorot_normal', activation='relu'))
# model.add(Dropout(0.5))
          
# model.add(Dense(128, kernel_initializer='glorot_normal', activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dropout(droprate))

model.add(Dense(48, kernel_initializer='glorot_normal', activation='relu'))
model.add(Dropout(0.5))
model.add(Dropout(droprate))
model.add(Dense(2, activation="softmax"))
model.summary() # Summary of the architecture

# Model Compile

In [ ]:
# Parameters Initialization
from tensorflow.keras.optimizers import Adam

#opt = rmsprop(lr=0.0001, decay=1e-6)

model.compile(loss="binary_crossentropy",optimizer=Adam(0.01), metrics=['binary_accuracy'])

# Train Custom Model

In [ ]:
from sklearn.metrics import confusion_matrix
import tensorflow as tf

class PredictionCallback(tf.keras.callbacks.Callback):    
    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(valid_flow[0][0])
        y_test = valid_flow[0][1]
        y_pred_labels = np.argmax(y_pred, axis=1)
        y_test_labels = np.argmax(y_test, axis=1)
        print(y_pred_labels.shape)
        print(y_test_labels.shape)
        cfm = confusion_matrix(y_test_labels, y_pred_labels)
        print(cfm)
        print(y_pred[0], y_test[0])

In [ ]:
# train_steps = 30000//batch_size
# valid_steps = 5000//batch_size
history = model.fit(
    train_flow,
    epochs=10,
#     steps_per_epoch = train_steps,
    validation_data = valid_flow,
#     validation_steps = valid_steps,
    callbacks=[PredictionCallback()]
)

In [ ]:
model.save("custom_model.h5") # save model

In [ ]:
"""
Plot the training and validation loss
epochs - list of epoch numbers
loss - training loss for each epoch
val_loss - validation loss for each epoch
"""
def plot_loss(epochs, loss, val_loss):
    plt.plot(epochs, loss, 'bo', label='Training Loss')
    plt.plot(epochs, val_loss, 'orange', label = 'Validation Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.show()
"""
Plot the training and validation accuracy
epochs - list of epoch numbers
acc - training accuracy for each epoch
val_acc - validation accuracy for each epoch
"""
def plot_accuracy(epochs, acc, val_acc):
    plt.plot(epochs, acc, 'bo', label='Training accuracy')
    plt.plot(epochs, val_acc, 'orange', label = 'Validation accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()
    plt.show()

In [ ]:
acc = history.history['binary_accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plot_loss(range(1, len(loss) + 1), loss, val_loss)
plot_accuracy(range(1, len(loss) + 1), acc, val_acc)

# Predict and Evaluate

In [ ]:
y_pred = model.predict(test_flow)

y_test = test_flow.classes

In [ ]:
y_pred_labels = np.argmax(y_pred, axis=1)

In [ ]:
# y_pred_labels = np.argmax(y_pred, axis=1)
# # y_test_labels = np.argmax(y_test, axis=1)
# print(y_pred_labels.shape)
# y_test.shape
# print(y_pred_labels[50:90], y_test[50:90])

In [ ]:
confusion_matrix(y_test, y_pred_labels)

In [ ]:
from sklearn import metrics
print("ROC AUC Score:", metrics.roc_auc_score(y_test, y_pred_labels))
print("AP Score:", metrics.average_precision_score(y_test, y_pred_labels))
print()
print(metrics.classification_report(y_test, y_pred_labels))